In [29]:
import pandas as pd
from sklearn.neighbors import KDTree
import numpy as np
import gensim
from sklearn.cluster import KMeans
import openpyxl

In [30]:
def read_csv_column(csv_path, column_name):
    df = pd.read_csv(csv_path, encoding='utf-8', low_memory=False)
    return df[column_name].tolist()

def get_top_words(index2word, k, word_centers, word_vectors):
    tree = KDTree(word_vectors)
# Closest points for each Cluster center is used to query the closest 20 points to it.
    closest_points = [tree.query(np.reshape(x, (1, -1)), k = k) for x in word_centers]
    closest_words_idxes = [x[1] for x in closest_points]
# Word Index is queried for each position in the above array, and added to a Dictionary.
    closest_words = {}
    for index in range(0, len(closest_words_idxes)):
        closest_words['Cluster #' + str(index)] = [index2word[j] for j in closest_words_idxes[index][0]]
# A DataFrame is generated from the dictionary.
    df = pd.DataFrame(closest_words)
    df.index = df.index + 1
    return df

In [31]:
contents = read_csv_column('../dataset/trump/trump01.csv', '微博正文')
documents = []
for content in contents:
    content = content.split()
    new_content = []
    for i in content:
        if len(i) != 1:
            new_content.append(i)
    documents.append(new_content)

In [32]:
model = gensim.models.Word2Vec (documents, size=150, window=10, min_count=2, workers=10)
# model.train(documents,total_examples=len(documents),epochs=10)

2021-04-09 21:23:01,835 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2021-04-09 21:23:01,836 : INFO : collecting all words and their counts
2021-04-09 21:23:01,837 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-04-09 21:23:01,905 : INFO : PROGRESS: at sentence #10000, processed 285434 words, keeping 24494 word types
2021-04-09 21:23:01,959 : INFO : collected 36603 word types from a corpus of 552662 raw words and 18398 sentences
2021-04-09 21:23:01,960 : INFO : Loading a fresh vocabulary
2021-04-09 21:23:01,996 : INFO : effective_min_count=2 retains 18856 unique words (51% of original 36603, drops 17747)
2021-04-09 21:23:01,996 : INFO : effective_min_count=2 leaves 534915 word corpus (96% of original 552662, drops 17747)
2021-04-09 21:23:02,040 : INFO : deleting the raw counts dictionary of 36603 items
2021-04-09 21:23:02,041 : INFO : sample=0.001 downsamples 23 most-common words
2021-04-09 21:23:02,042 : INFO : down

In [33]:
#获取model里面的说有关键词
vocab = model.wv.vocab
print(type(vocab))

<class 'dict'>


In [34]:
keys=model.wv.vocab.keys()

#获取词对于的词向量
word_vector=[]
for key in keys:
    word_vector.append(model[key])

<ipython-input-34-5209c2fa48f0>:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  word_vector.append(model[key])


In [35]:
#分类
kMeans_clustering = KMeans(n_clusters=10, init='k-means++')
clusters = kMeans_clustering.fit_predict(word_vector)
centers = kMeans_clustering.cluster_centers_
centroid_map = dict(zip(model.wv.index2word, clusters))
# s = clf.fit(word_vector)

In [36]:
Z = model.wv.syn0
top_words = get_top_words(model.wv.index2word, 20, centers, Z)

<ipython-input-36-4117ed6994a6>:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  Z = model.wv.syn0


In [37]:
top_words

,Cluster #0,Cluster #1,Cluster #2,Cluster #3,Cluster #4,Cluster #5,Cluster #6,Cluster #7,Cluster #8,Cluster #9
1,證券,维护,政治局,艺术,五角大楼,村庄,空间,苏莱曼,民主党人,梅拉妮亚
2,玩意,尊重,中共中央,绥靖主义,德黑兰,俊子,需求,卡西姆苏莱马尼,团队,口令
3,滚然而下,建立,委员,人性,伊斯兰,blog186,大幅,讲述,指控,宁静
4,爬起,目的,前往,打造,军方,寂寞,水平,数千,文件,出头
5,成年人,条件,28日,落后,军官,007,行业,口号,提交,Barron
6,趣味,各方,国务院,阶层,革命,海不扬波,财政,街头,共和党,大年初三
7,手臂,做出,全国,中文,高官,秋野,突破,哭泣,程序,箭头
8,taylors,进一步,30日,期權,首都,可可西,调整,遭遇,证人,略带
9,中外,寻求,牵头人,典型,指挥官,爽风,趋势,报仇,议长,破天荒
10,技術,缓和,10日,效果,复仇,斑马线,产业,攻破,参院,EXO
